In [46]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import pandas as pd
from nltk.corpus import wordnet as wn 
import csv
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [88]:
data = pd.read_csv("../Filtered Data/training.csv")

In [89]:
data.shape

(4920, 174)

In [90]:
data.head()

,Unnamed: 0,coma,cramps,sweating,skin_rash,rusty_sputum,mild_fever,excessive_hunger,cough,lack_of_concentration,...,Hyperthyroidism,Hypoglycemia,Osteoarthristis,Arthritis,(vertigo) Paroymsal Positional Vertigo,Acne,Urinary tract infection,Psoriasis,Impetigo,disease
0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Fungal infection
1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Fungal infection
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Fungal infection
3,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Fungal infection
4,4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Fungal infection


In [91]:
data.drop('Unnamed: 0',axis=1, inplace=True)

In [92]:
symp_new = data.columns

In [93]:
X= data[symp_new]
y = data[["disease"]]
np.ravel(y)
print(X)

      coma  cramps  sweating  skin_rash  rusty_sputum  mild_fever  \
0      0.0     0.0       0.0        1.0           0.0         0.0   
1      0.0     0.0       0.0        1.0           0.0         0.0   
2      0.0     0.0       0.0        0.0           0.0         0.0   
3      0.0     0.0       0.0        1.0           0.0         0.0   
4      0.0     0.0       0.0        1.0           0.0         0.0   
...    ...     ...       ...        ...           ...         ...   
4915   0.0     0.0       0.0        0.0           0.0         0.0   
4916   0.0     0.0       0.0        1.0           0.0         0.0   
4917   0.0     0.0       0.0        0.0           0.0         0.0   
4918   0.0     0.0       0.0        1.0           0.0         0.0   
4919   0.0     0.0       0.0        1.0           0.0         0.0   

      excessive_hunger  cough  lack_of_concentration  pain_behind_the_eyes  \
0                  0.0    0.0                    0.0                   0.0   
1              

In [95]:
objList = data.select_dtypes(include = "object").columns
print (objList)

Index(['disease'], dtype='object')


In [96]:
print (data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4920 entries, 0 to 4919
Columns: 173 entries, coma to disease
dtypes: float64(172), object(1)
memory usage: 6.5+ MB
None


In [97]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for feat in objList:
    data[feat] = le.fit_transform(data[feat].astype(str))

print (data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4920 entries, 0 to 4919
Columns: 173 entries, coma to disease
dtypes: float64(172), int32(1)
memory usage: 6.5 MB
None


In [98]:
symp=[]
disease=[]
for i in range(len(data)):
    symp.append(data.columns[data.iloc[i]==1].to_list())
    disease.append(data.iloc[i,-1])

In [99]:
symp[0]

['skin_rash',
 'nodal_skin_eruptions',
 'itching',
 'dischromic _patches',
 'Fungal infection']

In [100]:
all_symp_col=list(data.columns[:-1])
def clean_symp(sym):
    return sym.replace('_',' ').replace('.1','').replace('(typhos)','').replace('yellowish','yellow').replace('yellowing','yellow') 

In [101]:
all_symp=[clean_symp(sym) for sym in (all_symp_col)]

In [102]:
found_syns=[]
notFound_syns=[]
for sym in all_symp:
    if not wn.synsets(sym) :
        notFound_syns.append(sym)
    else:
        found_syns.append(sym)

In [103]:
print(f'Found disease in wordnet {len(found_syns)}')

Found disease in wordnet 52


In [104]:
print(f'Not found disease in wordnet {len(notFound_syns)}')

Not found disease in wordnet 120


#### CREATING KNeighborsClassifier for prediction

In [105]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.3, random_state=0,stratify=y)

In [106]:
y_train.head()

,disease
959,Paralysis (brain hemorrhage)
511,Hypertension
3765,Osteoarthristis
923,Hypertension
2010,Acne


In [107]:
disease = data[["disease"]]

In [108]:
X_train.head()

,coma,cramps,sweating,skin_rash,rusty_sputum,mild_fever,excessive_hunger,cough,lack_of_concentration,pain_behind_the_eyes,...,Hyperthyroidism,Hypoglycemia,Osteoarthristis,Arthritis,(vertigo) Paroymsal Positional Vertigo,Acne,Urinary tract infection,Psoriasis,Impetigo,disease
959,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32
511,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23
3765,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,31
923,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23
2010,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2


## Decision Tree

In [116]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
tree = DecisionTreeClassifier()
tree_params = [{'max_depth': list(range(5, 15)), 'criterion': ["entropy","gini"]}]
clf = GridSearchCV(tree, tree_params, cv = 10, scoring='accuracy')
clf.fit(X_train, y_train)
print(clf.best_params_)
print(clf.best_score_)

{'criterion': 'entropy', 'max_depth': 9}
1.0


In [122]:
decision_tree = DecisionTreeClassifier(max_depth=9,criterion="entropy")
decision_tree.fit(X_train,y_train)
print("Accuracy on training: {:.2f}".format(decision_tree.score(X_train, y_train)))
print("Accuracy on test: {:.2f}".format(decision_tree.score(X_test, y_test)))

Accuracy on training: 1.00
Accuracy on test: 1.00


In [124]:
print(classification_report(y_test,decision_tree.predict(X_test)))

                                         precision    recall  f1-score   support

(vertigo) Paroymsal  Positional Vertigo       1.00      1.00      1.00        36
                                   AIDS       1.00      1.00      1.00        36
                                   Acne       1.00      1.00      1.00        36
                    Alcoholic hepatitis       1.00      1.00      1.00        36
                                Allergy       1.00      1.00      1.00        36
                              Arthritis       1.00      1.00      1.00        36
                       Bronchial Asthma       1.00      1.00      1.00        36
                   Cervical spondylosis       1.00      1.00      1.00        36
                            Chicken pox       1.00      1.00      1.00        36
                    Chronic cholestasis       1.00      1.00      1.00        36
                            Common Cold       1.00      1.00      1.00        36
                           

## KNN

In [118]:
from sklearn.neighbors import KNeighborsClassifier  
knn = KNeighborsClassifier()
knn_params= {"n_neighbors":list(range(1, 20))}
clf4 = GridSearchCV(knn, knn_params, cv = 10, scoring='accuracy')
clf4.fit(X_train, y_train)
print(clf4.best_params_)
print(clf4.best_score_)

{'n_neighbors': 1}
1.0


In [121]:
knn_model_1 = KNeighborsClassifier(n_neighbors=1) 
knn_model_1.fit(X_train, y_train) 
print("Accuracy on training: {:.2f}".format(knn_model_1.score(X_train, y_train)))
print("Accuracy on test: {:.2f}".format(knn_model_1.score(X_test, y_test)))

Accuracy on training: 1.00
Accuracy on test: 1.00


In [110]:
print(classification_report(y_test,knn_model_1.predict(X_test)))

                                         precision    recall  f1-score   support

(vertigo) Paroymsal  Positional Vertigo       1.00      1.00      1.00        36
                                   AIDS       1.00      1.00      1.00        36
                                   Acne       1.00      1.00      1.00        36
                    Alcoholic hepatitis       1.00      1.00      1.00        36
                                Allergy       1.00      1.00      1.00        36
                              Arthritis       1.00      1.00      1.00        36
                       Bronchial Asthma       1.00      1.00      1.00        36
                   Cervical spondylosis       1.00      1.00      1.00        36
                            Chicken pox       1.00      1.00      1.00        36
                    Chronic cholestasis       1.00      1.00      1.00        36
                            Common Cold       1.00      1.00      1.00        36
                           